In [6]:
import torch
import pandas as pd
import numpy as np
from ultralytics import YOLO

| Git | Docs | Summary |
|:---:|:----:|:-------:|
| [`Ultralytics YOLOv8`](https://github.com/ultralytics/ultralytics) | [`YOLOv8 Docs`](https://docs.ultralytics.com/) | YOLOv7 for the masses |
| [`Comet Model Tracking`](https://www.comet.com/site/lp/yolov5-with-comet/?utm_source=yolov5&utm_medium=partner&utm_campaign=partner_yolov5_2022&utm_content=github) | `-` | Storage & updating software for model training |
| [`YOLOv7 WongKinYiu`](https://github.com/WongKinYiu/yolov7) | [`YOLOv7 Paper`](https://arxiv.org/abs/2207.02696) | Original YOLOv7 Repo |
| [`Roboflow Data`](https://app.roboflow.com/tid-detection/ionospheresegmentation/14) | `-` | Roboflow custom dataset for download |
| [`Norfair`](https://github.com/tryolabs/norfair) | [`Norfair Docs`](https://tryolabs.github.io/norfair/2.2/reference/) | Tracking for use with object detectors |

In [1]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [8]:
!nvidia-smi
!nvcc -V
print('Cuda Is Available:', end='')
print(torch.cuda.is_available())

Sat Jan 14 00:12:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:17:00.0 Off |                  N/A |
|  0%   34C    P8     6W / 260W |     13MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:18:00.0 Off |                  N/A |
|  0%   

In [ ]:
od_model = YOLO("yolov8x.pt")  # load a pretrained model (recommended for training)

# Use the model
results = od_model.train(data="data.yaml", epochs=10, imgsz=640)  # train the model
results = od_model.val()  # evaluate model performance on the validation set
results = od_model("tid_example.jpg")  # predict on an image
success = od_model.export(format="-")

  0%|          | 0.00/131M [00:00<?, ?B/s]


Ultralytics YOLOv8.0.5 🚀 Python-3.9.8 torch-1.10.2+cu102 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11020MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.yaml, data=data.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=True, dnn=False, plots=False, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, retina_masks=False, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, workspace=4, nms=

In [ ]:
is_model = YOLO("yolov8x-seg.pt")  # load a pretrained model (recommended for training)

# Use the model
results = is_model.train(data="data.yaml", epochs=10, imgsz=640)  # train the model
results = is_model.val()  # evaluate model performance on the validation set
results = is_model("tid_example.jpg")  # predict on an image
success = is_model.export(format="-")

In [ ]:
model = YOLO("path/to/best.pt")

# Norfair Integration Attempt

In [ ]:
import os

import numpy as np
import torch
import torchvision.ops.boxes as bops

import norfair
from norfair import Detection, Paths, Tracker, Video

DISTANCE_THRESHOLD_BBOX = 0.7
DISTANCE_THRESHOLD_CENTROID = 30
CLASSES = None
MAX_DISTANCE: int = 10000

In [ ]:
try:
    self.model = torch.hub.load("WongKinYiu/yolov7", "custom", model_path)
except:
    raise Exception("Failed to load model from {}".format(model_path))


In [ ]:
def yolo_detections_to_norfair_detections(yolo_detections: torch.tensor):
    norfair_detections = []
    detection_as_xyxys = yolo_detections.xyxy[0]
    for detection_as_xyxy in detections_as_xyxys:
        bbox = np.array([
            [detection_as_xyxy[0].item(), detection_as_xyxy[1].item()],
            [detection_as_xyxy[2].item(), detection_as_xyxy[3].item()],
        ])
        scores = np.array([detection_as_xyxy[4].item(), detection_as_xyxy[4].item()])
        norfair_detections.append(Detection(points=bbox, scores=scores, label=int(detection_as_xyxy[-1].item())))

    return norfair_detections

In [ ]:
for file in files:
    video = Video(input_path=file)

    tracker = Tracker(
        distance_function='iou',
        distance_threshold=DISTANCE_THRESHOLD_BBOX,
    )

    for frame in video:
        yolo_detections = model(
            frame,
            conf_threshold=args.conf_threshold,
            iou_threshold=args.iou_threshold,
            image_size=args.img_size,
            classes=CLASSES,
        )
        detections = yolo_detections_to_norfair_detections(
            yolo_detections, 
#             track_points=args.track_points,
        )
        tracked_objects = tracker.update(detections=detections)
        if args.track_points == "centroid":
            norfair.draw_points(frame, detections)
            norfair.draw_tracked_objects(frame, tracked_objects)
        elif args.track_points == "bbox":
            norfair.draw_boxes(frame, detections)
            norfair.draw_tracked_boxes(frame, tracked_objects)
        video.write(frame)